In [1]:
from msal import ConfidentialClientApplication
import pandas as pd
import os
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential
import logging, time

def_credential = DefaultAzureCredential()  

## connect to Azure Key Vault securely
keyVaultName = os.environ["KEY_VAULT_NAME"]
KVUri = f"https://{keyVaultName}.vault.azure.net"
kv_client = SecretClient(vault_url=KVUri, credential=def_credential)

## Secret Keys for Bolddesk
api_key       = kv_client.get_secret('infosec-api-key').value

## Secret Keys for Azure App Registration
tenant_id = kv_client.get_secret('azure-tenant-id').value
client_id = kv_client.get_secret('idgov-app-client-id').value
client_secret = kv_client.get_secret('idgov-app-client-secret').value

In [2]:
import sys
sys.path.append('../')
# from module.azure_ad import AzureAD
from module.infosec import Infosec 
from module.warehouse import Warehouse
from module.azure_ad import AzureAD

## Initialize AD and Bolddesk API Module
url = 'https://securityiq-eu.infosecinstitute.com/api/v2'
ad = AzureAD(tenant_id, client_id, client_secret)
ifs = Infosec(url, api_key)

## initialize warehouse
wh = Warehouse(
    server=os.environ["DB_SERVER"],
    database=os.environ["DB_NAME"],
    credential=def_credential
)

In [3]:
ifs.list_timeline_events()

[{'id': 'eu8Q6',
  'type': 'created-learner',
  'timestamp': '2023-03-29T09:15:22+00:00',
  'learner_id': 'TE6y',
  'campaign_id': None,
  'quarantined_email_id': None,
  'module_id': None,
  'campaign_run_id': None,
  'template_id': None,
  'learner_group_id': None,
  'assessment_id': None},
 {'id': 'wW3U1',
  'type': 'created-learner',
  'timestamp': '2023-04-12T05:19:40+00:00',
  'learner_id': 'TsUM',
  'campaign_id': None,
  'quarantined_email_id': None,
  'module_id': None,
  'campaign_run_id': None,
  'template_id': None,
  'learner_group_id': None,
  'assessment_id': None},
 {'id': 'wW3Us',
  'type': 'added-to-group',
  'timestamp': '2023-04-12T05:19:40+00:00',
  'learner_id': 'TsUM',
  'campaign_id': None,
  'quarantined_email_id': None,
  'module_id': None,
  'campaign_run_id': None,
  'template_id': None,
  'learner_group_id': None,
  'assessment_id': None},
 {'id': 'wW3UV',
  'type': 'created-learner',
  'timestamp': '2023-04-12T05:19:40+00:00',
  'learner_id': 'TsUa',
  'ca

In [8]:
df = pd.DataFrame()
cam_runs     = ifs.list_awareness_campaigns_runs()
for cr in cam_runs:
    run_ids = [r.get('id') for r in cr.get('runs')]
    campaign_id = cr.get('campaign').get('id')
    print(campaign_id)
    temp_df = ifs.list_campaignRunsLearners(campaign = campaign_id, runs = run_ids)
    df = pd.concat([df, temp_df], axis=0)

GlV
GSZ
GSj
GSy
GiT
YIM


C:\Users\User\AppData\Local\Temp\ipykernel_2540\3211662414.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp_df], axis=0)


csn


In [9]:
cam_runs

[{'campaign': {'id': 'GlV',
   'name': 'NERA Cybersecurity Awareness Training 2023',
   'type': 'awareness',
   'running': False},
  'runs': [{'id': 'cWr',
    'campaign_id': 'GlV',
    'start': '2023-07-09T16:00:00+00:00',
    'end': '2023-07-15T16:00:00+00:00',
    'run_number': 1}]},
 {'campaign': {'id': 'GSZ',
   'name': 'NERA Cybersecurity Awareness Training 2023 part 1',
   'type': 'awareness',
   'running': False},
  'runs': [{'id': 'c0i',
    'campaign_id': 'GSZ',
    'start': '2023-07-17T16:00:00+00:00',
    'end': '2023-08-11T16:00:00+00:00',
    'run_number': 1}]},
 {'campaign': {'id': 'GSj',
   'name': 'NERA Cybersecurity Awareness Training 2023 part 2',
   'type': 'awareness',
   'running': False},
  'runs': [{'id': 'c0u',
    'campaign_id': 'GSj',
    'start': '2023-07-17T16:00:00+00:00',
    'end': '2023-08-11T16:00:00+00:00',
    'run_number': 1}]},
 {'campaign': {'id': 'GSy',
   'name': 'NERA Cybersecurity Awareness Training 2023  part 3',
   'type': 'awareness',
   'r

In [7]:
campaign_id = 'GiT'


,learner_id,email,first_name,last_name,campaign_id,run_id,status,completed_on
0,TE6y,kehsoon.yong@nera.net,Keh Soon,Yong,GlV,cWr,started,NaT
1,Tsfi,jacqueline.tan@nera.net,Jacqueline,Tan,GlV,cWr,not_started,NaT
2,Tsfu,alex.loh@nera.net,Alex,Loh,GlV,cWr,not_started,NaT
3,TsLn,hoiming.chong@nera.net,Hoi Ming,Chong,GlV,cWr,completed,2023-08-18 07:23:24
4,TsLj,aaron.lim@nera.net,Aaron,Lim,GlV,cWr,completed,2023-07-31 09:31:04
...,...,...,...,...,...,...,...,...
419,AXLw,muhammad.haroon@nera.net,Muhammad,Haroon,YIM,148,not_started,NaT
420,A6lq,christopher.yong@nera.net,Christopher,Yong,YIM,148,completed,2024-03-04 09:02:02
421,A6lb,hueyshieng.chua@nera.net,Huey Shieng,Chua,YIM,148,started,NaT
0,TsLe,soonkuan.kua@nera.net,Kua,Soon Kuan,csn,M4J,started,NaT


In [17]:
ifs.list_campaignRunsLearners(campaign = campaign_id, runs = run_ids)

KeyError: 'learner_id'

In [9]:
ifs.list_campaignRunsLearners('YIM',[148])

,learner_id,email,first_name,last_name,campaign_id,run_id,status,completed_on
0,TE6y,kehsoon.yong@nera.net,Keh Soon,Yong,YIM,148,completed,2023-09-12 09:11:34
1,TsUM,erwin.novriyanto@nera.net,Erwin,Novriyanto,YIM,148,completed,2023-09-13 06:47:32
2,TsUa,minmyatoo@nera.net,Mike,Min Myat Oo,YIM,148,not_started,NaT
3,TsU6,alan.ng@nera.net,Alan,Ng,YIM,148,not_started,NaT
4,TsU8,alfie.ong@nera.net,Alfie,Ong,YIM,148,not_started,NaT
...,...,...,...,...,...,...,...,...
428,rTOT,hammad.paracha@nera.net,Hammad Ali,Paracha,YIM,148,not_started,NaT
429,rTf8,mark.lumame@nera.net,Mark,Jay Lumame Abueme,YIM,148,completed,2023-09-12 09:28:42
430,rrDD,rahmatullah@nera.net,Rahmatullah,Nurhidayat,YIM,148,not_started,NaT
431,rroW,hweeyee.sng@nera.net,Hwee Yee,Sng,YIM,148,not_started,NaT


In [19]:
cols = ['learner_id', 'email', 'first_name', 'last_name', 'campaign_id', 'run_id', 'status', 'completed_on']
df = pd.DataFrame(columns=cols)

